In [33]:
from optbinning import OptimalBinning


from os import listdir
from os.path import isfile, join
from datetime import datetime, timedelta
import time
import sys

import pandas as pd
import numpy as np
import random

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

from optbinning import OptimalBinning
from catboost import CatBoostClassifier, Pool, cv

import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


In [34]:

i = 'dataset1.csv'
df = pd.read_csv(i, sep=';', low_memory=False, error_bad_lines=False)

In [35]:
sum(df.id.value_counts()>1) #no repeated rows

0

In [36]:
prop = df.default.value_counts()
prop[1]/(prop[0]+prop[1])
sum(df.default.isnull()) # to predict

10000

## Analize distributions from previous data vs prediction.


In [37]:
print(df.shape)
#df.columns
df_types = df.dtypes
df_types.value_counts()

(99976, 43)


int64      20
float64    19
object      3
bool        1
dtype: int64

In [46]:
categorical_cols = df_types[df_types=='object']
categorical_cols = list(categorical_cols.index)
categorical_cols = [x for x in categorical_cols if x!='uuid']
print(categorical_cols)

numerical_cols = df_types[df_types!='object']
numerical_cols = list(numerical_cols.index)
numerical_cols = [x for x in numerical_cols if x!='default']


numerical_cols = list(set(numerical_cols)-set(ordinal_cols))

['x_13', 'x_14', 'x_18']


In [47]:
random.seed(1)
df['random'] = np.random.randint(1, df.shape[0], df.shape[0])/df.shape[0]

In [48]:
df_hist = df.copy()
df_hist = df_hist[df_hist.default.isnull()==False]
display(df_hist.default.value_counts())

df_pred = df.copy()
df_pred = df_pred[df_pred.default.isnull()]
df_pred.default.value_counts()
df_pred.shape

0.0    88688
1.0     1288
Name: default, dtype: int64

(10000, 44)

In [49]:
df_hist['random_num'] = np.random.choice(df_hist.shape[0], df_hist.shape[0], replace=False)
sum(df_hist['random_num'].value_counts()>1)

0

## changes in distributions of features history vs data to predict

In [50]:
dist_num = {}
tblh = df_hist[df_hist.random_num< df_pred.shape[0]].copy()
tblp = df_pred.copy()
print(tblh.shape)
for var in numerical_cols + ordinal_cols:
    idx_null_hist = pd.isnull(tblh[var])
    idx_null_pred = pd.isnull(tblp[var])
    q_nulls_hist = sum(idx_null_hist)
    q_nulls_pred = sum(idx_null_pred)
    uniq_val_hist = len(set(tblh[ idx_null_hist == False ][var]))
    uniq_val_pred = len(set(tblp[ idx_null_pred == False ][var]))
    mean_hist = np.mean(tblh[ idx_null_hist == False ][var])
    mean_pred = np.mean(tblp[ idx_null_pred == False ][var])
    ks_test = stats.ks_2samp(tblh[var], tblp[var])
    dist_num[var] = [uniq_val_hist, uniq_val_pred, q_nulls_hist, q_nulls_pred, mean_hist, mean_pred, ks_test[0], ks_test[1]]
    

dif_dist_num =  (pd.DataFrame.from_dict(dist_num)).transpose()
dif_dist_num.columns = ['uniq_val_hist', 'uniq_val_pred', 'q_nulls_hist', 'q_nulls_pred', 'mean_hist', 'mean_pred','ks_test', 'p_value']

dif_dist_num = dif_dist_num.sort_values('ks_test')
idx_m = abs(dif_dist_num.mean_hist - dif_dist_num.mean_pred) /dif_dist_num.mean_hist >.2 
idx_k = dif_dist_num.ks_test>0.03
idx_p = abs(dif_dist_num.q_nulls_hist - dif_dist_num.q_nulls_pred) /dif_dist_num.q_nulls_hist >.2 
idx_u = abs(dif_dist_num.uniq_val_hist - dif_dist_num.uniq_val_pred) /dif_dist_num.uniq_val_hist >.5


very_dif  = dif_dist_num[ idx_m | idx_k| idx_p| idx_u]
display (very_dif)

maybe_exclude_num = list(very_dif.index)


(10000, 45)


,uniq_val_hist,uniq_val_pred,q_nulls_hist,q_nulls_pred,mean_hist,mean_pred,ks_test,p_value
x_35,15.0,17.0,0.0,0.0,3.2195,3.954700,0.0005,1.0
x_27,1.0,3.0,1804.0,1773.0,0.0000,0.000365,0.0031,1.0
x_26,1.0,2.0,1804.0,1773.0,0.0000,0.000243,0.0031,1.0
id,10000.0,10000.0,0.0,0.0,44838.5829,94975.500000,1.0000,0.0


In [54]:
dist_cat = {}
tblh = df_hist[df_hist.random_num< df_pred.shape[0]].copy()
tblp = df_pred.copy()
print(tblh.shape)
for var in categorical_cols  :
    
    idx_null_hist = pd.isnull(tblh[var])
    idx_null_pred = pd.isnull(tblp[var])
    q_nulls_hist = sum(idx_null_hist)
    q_nulls_pred = sum(idx_null_pred)
    uniq_val_hist = len(set(tblh[ idx_null_hist == False ][var]))
    uniq_val_pred = len(set(tblp[ idx_null_pred == False ][var]))

    tblh.loc[tblh[var].isnull(),var]='SD'
    tblh.loc[tblh[var].isna(),var]='SD'
    tblp.loc[tblp[var].isnull(),var]='SD'
    tblp.loc[tblp[var].isna(),var]='SD'

    dh = tblh[var].value_counts().reset_index()
    dh.columns = ['levels', str(var)+'_h' ]
    dp = tblp[var].value_counts().reset_index()
    dp.columns = ['levels', str(var)+'_p' ]
    all_levels = (set(dh.levels)).union( set(dp.levels))
    all_levels_df = pd.DataFrame(index=all_levels).reset_index()
    all_levels_df.columns = ['levels']
    res = pd.merge(pd.merge( all_levels_df, dh, how= 'left') , dp, how ='left')
    table = [list(res[str(var)+'_h']) , list(res[str(var)+'_p'])]
    stat, p, dof, expected = chi2_contingency(table)
    dist_cat[var] = [uniq_val_hist, uniq_val_pred, q_nulls_hist, q_nulls_pred, stat, p]
    
dif_dist_cat =  (pd.DataFrame.from_dict(dist_cat)).transpose()
dif_dist_cat.columns = ['uniq_val_hist', 'uniq_val_pred', 'q_nulls_hist', 'q_nulls_pred', 'statxi2', 'pxi2']

dif_dist_cat = dif_dist_cat.sort_values('pxi2')
idx_x = dif_dist_cat.pxi2<0.2
idx_p = abs(dif_dist_cat.q_nulls_hist - dif_dist_cat.q_nulls_pred) /dif_dist_cat.q_nulls_hist >.2 
idx_u = abs(dif_dist_cat.uniq_val_hist - dif_dist_cat.uniq_val_pred) /dif_dist_cat.uniq_val_hist >.5

#display(dif_dist_cat)
very_dif  = dif_dist_cat[ idx_x| idx_p| idx_u]
display (very_dif)

maybe_exclude_cat = list(very_dif.index)

(10000, 45)


,uniq_val_hist,uniq_val_pred,q_nulls_hist,q_nulls_pred,statxi2,pxi2


#### view specific variable

In [60]:
var = 'x_35'

display(df_hist.groupby(var).agg({'id': 'count', 'default': 'sum'}))
df_pred.groupby(var).agg({'id': 'count', 'default': 'sum'})

,id,default
x_35,,
0,89774,1261.0
47,1,0.0
90,1,0.0
99,1,0.0
348,2,0.0
...,...,...
7200,2,0.0
7910,1,0.0
11190,1,0.0


,id,default
x_35,,
0,9980,0.0
400,1,0.0
500,5,0.0
695,1,0.0
1000,1,0.0
1032,1,0.0
1275,1,0.0
1476,1,0.0
2080,1,0.0
